# Теоретическая часть

## 1. Наивный байесовский классификатор, модель Бернулли

### A)
**Чему равна вероятность $P(w_i ∈ d | d ∈ c_j)$ встретить $i$-ое слово из словаря в случайном
документе класса $c_j$?**

Переформулируем этот вопрос для модели Бернулли: \
**Какова вероятность, что i-ый признак документа $d$ равен 1, при условии, что документ принадлежит классу $c_j$.**
\
$P(f_i(d) = 1 | d \in c_j)$, где $f_i$ - функция, возвращающая $i$-ый признак для документа $d$.\
\
По формуле условной вероятности: $$P(f_i(d) = 1 | d \in c_j) = \frac{P(f_i(d) = 1, d \in c_j)}{P(d \in c_j)}$$
Оценим эту вероятность на выборке.\
Это можно сделать по частоте, с которой признак $f_i(d) = 1$ во всех документах выборки.
$$P(w_i | c_j) = P(f_i(d) = 1 | d \in c_j) = \frac{cnt(f_i(d) = 1, d \in c_j)}{\sum_{x \in {0,1}} cnt(f_i(d) = x, d \in c_j)}$$
$cnt(cond)$ - количество документов из выборки, удовлетворяющих условию $cond$.

Если в тестовой выборки нам встретится слово, которое не встречалалось в обучающей, то вероятность $P(w_i | c_j)$ обнулится для всех $c_j$ и мы не сможем предсказать класс предложения. 
Поэтому применим сглаживание Лапласа, т.е. добавим фиктивные $\alpha$ появлений каждого слова в каждом классе.

$$P(w_i | c_j) = P(f_i(d) = 1 | d \in c_j) = \frac{cnt(f_i(d) = 1, d \in c_j) + \alpha}{\sum_{x \in {0,1}} (cnt(f_i(d) = x, d \in c_j) + \alpha)} = \frac{cnt(f_i(d) = 1, d \in c_j) + \alpha}{\sum_{x \in {0,1}} (cnt(f_i(d) = x, d \in c_j)) + 2*\alpha}$$
Обычно параметр $\alpha$ выбирается равным 1.

### B)
**Вывести $P(d=(k_1, k_2, ... , k_M) | d \in c_j)$ – вероятность того, что случайный документ $d$, принадлежащий классу $c_j$, будет состоять из $k_1, k_2, ... , k_M$ вхождений слов $v_1, v_2, ... , v_M$?**

В модели Бернулли мы учитываем только наличие слова в предложении. Поэтому теряем информацию о порядке слов и количестве вхождений каждого слова и значит $k_i = 0...1$

Из наивного предположения о независимости аргументов получаем: 
$$P(d=(k_1, k_2, ... , k_M) | d \in c_j) = \prod_{i=1}^{M} P(cnt(w_i) = k_i | d \in c_j)$$
Т.е. произведение вероятностей по каждому слову $w_i$, что оно встречается $k_i$ раз в случайном документе $d$ из класса $c_j$.

Посчитаем одну вероятность $P(cnt(w_i) = k_i | d \in c_j)$
$$P(cnt(w_i) = k_i | d \in c_j) = k_i * P(w_i | c_j) + (1 - k_i) * (1 - P(w_i | c_j))$$

Тогда исходная вероятность равна:
$$P(d=(k_1, k_2, ... , k_M) | d \in c_j) = \prod_{i=1}^{M}\left[k_i * P(w_i | c_j) + (1 - k_i) * (1 - P(w_i | c_j))\right]$$

### C)
**Вывести вероятность $P(c_j| d)$, что данный документ принадлежит классу $c_j$.**

Соберем вместе предыдущие пункты.
Для начала применим формулу Байеса. $P(c_j|d) = \frac{P(d|c_j)P(c_j)}{P(d)}$.
В ней нам неизвестны $P(c_j)$ и $P(d)$.\
$P(c_j)$ - вероятность встретить среди всех документов документ класса $c_j$.\
$P(d)$ - вероятность встретить документ $d$ среди всех документов.\
$P(c_j)$ мы можем оценить из обучающей выборки. $P(c_j) = \frac{N_{c_j}}{N}$, где $N_{c_j}$ - количество документов класса $c_j$, $N$ - общее количество документов.\
Для модели Бернулли, где каждый элемент задаётся M-мерным вектором из 0/1 и без учета порядка, мы можем посчитать $P(d) = \frac{N_d}{N}$, где $N_d$ - количество векторов $d$ среди всех векторов предложений.\
Но т.к. для каждого класса эта величина одинаковая, то для выявления максимальной вероятности она не нужна. Поэтому для классификации она не используется.

В итоге получаем оценку вероятности:
$$P(c_j|d) = P(c_j) * \prod_{i=1}^{M}\left[k_i * P(w_i | c_j) + (1 - k_i) * (1 - P(w_i | c_j))\right]$$

### D)
**Какой класс $c_j$ будет будет выдан для документа d классификатором, если предположить, что $P(c_j)$ и $P(d | c_j)$ заданы? Как можно оценить вероятность ошибки?**

При заданных вероятностях $P(c_j)$ и $P(d|c_j)$ мы можем посчитать все вероятности $P(c_j | d)$ и выбрать максимальную.\
$$c_j = argmax_{j} P(c_j|d) = argmax_{j} \left[P(d|c_j) * P(c_j)\right]$$

При этом, т.к. в $P(d | c_j)$ нам нужно счиать много произведений, то можно выбирать максимум из логарифмов этих величин.

Вероятность ошибки можно оценить как 
$$1 - \mathbb{E}(max_j(P(d | c_j)))$$

## 2. Наивный байесовский классификатор, мультиномиальная модель.

### A)
**Чему равна вероятность $P(w_i ∈ d | d ∈ c_j)$ встретить $i$-ое слово из словаря в случайном
документе класса $c_j$?**

В мультиномиальной модели мы рассматриваем документ, как вектор длины словаря, в котором указывается, сколько раз каждое слово встерчалось в предложении.

Поэтому оценивать вероятность встретить слово мы будем немного по другому, учитывая количество вхождений всех слов в документы.

$$P(w_i \in d | d \in c_j) = \frac{P(w_i \in d, d \in c_j)}{P(d \in c_j)}$$

Оценка будет выглядеть следующим образом.
$$P(w_i | c_j) = \frac{cnt(f_i(d), d \in c_j) + \alpha}{\sum_{i}\sum_{x} (cnt(f_i(d) = x, d \in c_j) + \alpha)} = \frac{cnt(f_i(d), d \in c_j) + \alpha}{\sum_{i}\sum_{x} (cnt(f_i(d) = x, d \in c_j)) + |V| * \alpha}$$ 

### B)
**Вывести $P(d=(k_1, k_2, ... , k_M) | d \in c_j)$ – вероятность того, что случайный документ $d$, принадлежащий классу $c_j$, будет состоять из $k_1, k_2, ... , k_M$ вхождений слов $v_1, v_2, ... , v_M$?**

Учитывая наивное предположение о независимости аргументов, мы можем рассматривать слова, как случайные величины с полиномиальным распределением, и тогда
$$P(d = (k_1, k_2, ..., k_m) | d \in c_j) = P(|d|) \frac{|d|!}{k_1!k_2!...k_m!} \prod_{i=1}^{m}P(w_i|c_j)^{k_i}$$

### C)
**Вывести вероятность $P(c_j| d)$, что данный документ принадлежит классу $c_j$.**

Тут аналогично предыдущему пункту после применения формулы Байеса получаем $P(c_j|d) = \frac{P(d|c_j)P(c_j)}{P(d)}$. \
Оценка для $P(c_j) = \frac{N_{c_j}}{N}$, где $N_{c_j}$ - количество документов класса $c_j$, $N$ - общее количество документов. \
$P(d)$ можно не оценивать, потому что для классификации не используется.\
Множитель с факториалами не зависит от класса, значит его тоже можно не использовать при классификации.\
Также будем считать, что длины документов встречаются с равной вероятностью.\
И тогда для классификации можно использовать оценку вероятности: $$P(c_j|d) = P(c_j) \prod_{i=1}^{m}P(w_i|c_j)^{k_i}$$

### D)
**Какой класс $c_j$ будет будет выдан для документа d классификатором, если предположить, что $P(c_j)$ и $P(d | c_j)$ заданы? Как можно оценить вероятность ошибки?**

Тут всё так же, как и в модели Бернулли.
$$c_j = argmax_{j} P(c_j|d) = argmax_{j} \left[P(d|c_j) * P(c_j)\right]$$


Вероятность ошибки можно оценить как 
$$1 - \mathbb{E}(max_j(P(d | c_j)))$$

# Практическая часть

## 1. Реализуем два классификатора и сравниваем их на FILMDB датасете.

**а)** Загрузить обучающую выборку в 2 списка – позитивные и негативные отзывы. Чему
равна минимальная, максимальная, средняя, медианная длина (в символах) позитивных /
негативных отзывов?

In [1]:
import pandas as pd
import re
from collections import defaultdict
import sys
import numpy as np
from math import log

pd.set_option('precision', 5)

In [19]:
train_texts_path = "./filimdb_evaluation/FILIMDB/train.texts"
train_labels_path = "./filimdb_evaluation/FILIMDB/train.labels"

dev_texts_path = "./filimdb_evaluation/FILIMDB/dev.texts"
dev_labels_path = "./filimdb_evaluation/FILIMDB/dev.labels"

test_texts_path = "./filimdb_evaluation/FILIMDB/test.texts"
test_labels_path = "./filimdb_evaluation/FILIMDB/test.labels"

with open(train_texts_path, 'r', encoding='utf-8',) as inp:
    train_data = list(map(str.strip, inp.readlines()))
with open(train_labels_path, 'r', encoding='utf-8',) as inp:
    train_labels = list(map(str.strip, inp.readlines()))

train_pos = [train_data[i] for i, lab in enumerate(train_labels) if lab == 'pos']
train_neg = [train_data[i] for i, lab in enumerate(train_labels) if lab == 'neg']
print(len(train_pos), len(train_neg))

7520 7480


In [20]:
def dataset_info(dataset):
    lens = [len(text) for text in dataset]
    min_len = min(lens)
    max_len = max(lens)
    mean_len = sum(lens) / len(lens)
    median_len = sorted(lens)[len(lens) // 2]

    print(f"min_len = {min_len}")
    print(f"max_len = {max_len}")
    print(f"mean_len = {mean_len}")
    print(f"median_len = {median_len}")
    
print("Positive: ")
dataset_info(train_pos)
print()
print("Negative: ")
dataset_info(train_neg)

Positive: 
min_len = 70
max_len = 10363
mean_len = 1360.801329787234
median_len = 997

Negative: 
min_len = 52
max_len = 8969
mean_len = 1316.356550802139
median_len = 981


**b)** Сделать предобработку. Перевести отзывы в нижний регистр. Вставить пробелы вокруг
всех символов, не являющихся цифрами или буквами.

In [21]:
print(train_pos[0])

I gave this movie a 10 because it needed to be rewarded for its scary elements and actors AND my god the enging! The thing is I don't want to tell anyone anything about the acting or story because it will ruin the movie. But I will recommend that you go straight to your nearest moviestore right now and rent it! (Don't forget popcorn!)


In [22]:
def preprocessing(text):
    text = text.lower()
    remove_tags = re.compile('<.*?>')
    text = re.sub(remove_tags, '', text)
    text = ''.join(sym if (sym.isalnum() or sym in (" ", "'")) else f" {sym} " for sym in text)
    return text


# print(preprocessing(train_neg[0]))
train_neg = list(map(preprocessing, train_neg))
train_pos = list(map(preprocessing, train_pos))

In [23]:
print(train_pos[0])

i gave this movie a 10 because it needed to be rewarded for its scary elements and actors and my god the enging !  the thing is i don't want to tell anyone anything about the acting or story because it will ruin the movie .  but i will recommend that you go straight to your nearest moviestore right now and rent it !   ( don't forget popcorn !  ) 


**c)** Сделать токенизацию – то есть представить каждый отзыв в виде списка токенов (слов,
чисел, знаков пунктуации). 

In [24]:
def tokenize_dataset(dataset):
    """
        arg: list(np.array) of texts
        return: np.array of tokenized texts(np.array)
    """
    tokenizer = re.compile(r"-?\d*[.,]?\d+|[?'\w]+|\S", re.MULTILINE | re.IGNORECASE)
    return list(map(lambda doc: list(tokenizer.findall(doc)), dataset))

In [25]:
train_pos = tokenize_dataset(train_pos)
train_neg = tokenize_dataset(train_neg)

In [26]:
print(train_pos[0])

['i', 'gave', 'this', 'movie', 'a', '10', 'because', 'it', 'needed', 'to', 'be', 'rewarded', 'for', 'its', 'scary', 'elements', 'and', 'actors', 'and', 'my', 'god', 'the', 'enging', '!', 'the', 'thing', 'is', 'i', "don't", 'want', 'to', 'tell', 'anyone', 'anything', 'about', 'the', 'acting', 'or', 'story', 'because', 'it', 'will', 'ruin', 'the', 'movie', '.', 'but', 'i', 'will', 'recommend', 'that', 'you', 'go', 'straight', 'to', 'your', 'nearest', 'moviestore', 'right', 'now', 'and', 'rent', 'it', '!', '(', "don't", 'forget', 'popcorn', '!', ')']


Важно, делать одинаковую предобработку, чтобы не появлялся неожиданный мусор в виде знаков препинания и других любых знаков, такой как слова с приклееными запятыми. 

**d)** Построить 2 Питоновских словаря слово→частота с частотами каждого слов в
позитивных и негативных отзывах. Для позитивных и негативных отзывов распечатайте по
30 самых частотных слов и их частоты. Распечатайте 30 слов с максимальными и 30 слов с
минимальными наивными байесовскими весами 

In [27]:
word_to_ind = {}
ind_to_word = {}
free_ind = 0

print(len(train_pos), len(train_neg))

pos_words = defaultdict(int)
word_in_pos_text = defaultdict(int)
neg_words = defaultdict(int)
word_in_neg_text = defaultdict(int)
cnt_pos = 0
cnt_neg = 0
for text in train_pos:
    been = set()
    for word in text:
        if word not in word_to_ind:
            word_to_ind[word] = free_ind
            ind_to_word[free_ind] = word
            free_ind += 1
        if word not in been:
            ind = word_to_ind[word]
            word_in_pos_text[ind] += 1
            been.add(word)
        pos_words[word_to_ind[word]] += 1
        cnt_pos += 1
       
bigrams = False

if bigrams:
    for text in train_pos:
        been = set()
        bigrams = [b for b in zip(text[:-1], text[1:])]
        for word in bigrams:
            if word not in word_to_ind:
                word_to_ind[word] = free_ind
                ind_to_word[free_ind] = word
                free_ind += 1
            if word not in been:
                ind = word_to_ind[word]
                word_in_pos_text[ind] += 1
                been.add(word)
            pos_words[word_to_ind[word]] += 1
            cnt_pos += 1
    
for text in train_neg:
    been = set()
    for word in text:
        if word not in word_to_ind:
            word_to_ind[word] = free_ind
            ind_to_word[free_ind] = word
            free_ind += 1
        if word not in been:
            ind = word_to_ind[word]
            word_in_neg_text[ind] += 1
            been.add(word)
        neg_words[word_to_ind[word]] += 1
        cnt_neg += 1

if bigrams:
    for text in train_neg:
        been = set()
        bigrams = [b for b in zip(text[:-1], text[1:])]
        for word in bigrams:
            if word not in word_to_ind:
                word_to_ind[word] = free_ind
                ind_to_word[free_ind] = word
                free_ind += 1
            if word not in been:
                ind = word_to_ind[word]
                word_in_neg_text[ind] += 1
                been.add(word)    
        neg_words[word_to_ind[word]] += 1
        cnt_neg += 1

N_words = len(word_to_ind)
N_words

7520 7480


71104

In [28]:
def find_popular(words_cnt, ind_to_word, top=30):
    words = sorted(list(words_cnt.items()), key=lambda p:(p[1]))
    top_words = words[-top:][::-1]
    return pd.DataFrame({'word' : [ind_to_word[ind] for ind,cnt in top_words], 'cnt' : [cnt for ind,cnt in top_words]})

In [29]:
print("Popular positive: ")
find_popular(pos_words, ind_to_word)

Popular positive: 


,word,cnt
0,the,105207
1,.,96930
2,",",87384
3,and,54478
4,a,50812
5,of,46517
6,to,40564
7,is,34866
8,in,30680
9,it,23892


In [30]:
print("Popular negative: ")
find_popular(neg_words, ind_to_word)

Popular negative: 


,word,cnt
0,.,101200
1,the,98927
2,",",79786
3,a,47958
4,and,45050
5,of,41743
6,to,41641
7,is,30346
8,in,26473
9,this,24527


In [31]:
best_bayes_weights = []
worst_bayes_weights = []
all_words = set(pos_words.keys())
for neg_w in neg_words.keys():
    all_words.add(neg_w)

eps = 1e-6

all_inds = list(all_words)
all_words_neg = [neg_words[word] for word in all_inds]
all_words_pos = [pos_words[word] for word in all_inds]
all_words = [ind_to_word[ind] for ind in all_inds]

d = {'word':all_words, 'neg_cnt':all_words_neg, 'pos_cnt':all_words_pos}

stats = pd.DataFrame(data=d)

V = len(all_words)

stats['bayes_weight'] = np.log(((stats['pos_cnt'] + 1) / (cnt_pos + V)) / ((stats['neg_cnt'] + 1) / (cnt_neg + V)))
stats.head()

,word,neg_cnt,pos_cnt,bayes_weight
0,i,24267,21922,-0.12769
1,gave,370,332,-0.13413
2,this,24527,21059,-0.17851
3,movie,14684,11585,-0.26310
4,a,47958,50812,0.03174


In [32]:
print("BEST BAYES WEIGHTS STATS: ")
stats.nlargest(30, 'bayes_weight')

BEST BAYES WEIGHTS STATS: 


,word,neg_cnt,pos_cnt,bayes_weight
12160,edie,0,71,4.25060
19724,antwone,0,69,4.22243
6880,gundam,0,61,4.10107
13685,paulie,0,61,4.10107
23028,mildred,0,60,4.08481
5974,corbett,0,44,3.78060
10900,din,0,42,3.73513
27025,flavia,0,39,3.66281
7571,biko,0,37,3.61152
15156,deathtrap,0,35,3.55745


Слова с большим наивным байсовским весом должны быть сильно позитивными.\
Из аномалий среди первых 30 слов можно выделить несколько слов: deathtrap - смертельная ловушка, ossessione-Обсессия

In [33]:
print("WORST BAYES WEIGHTS STATS: ")
stats.nsmallest(30, 'bayes_weight')

WORST BAYES WEIGHTS STATS: 


,word,neg_cnt,pos_cnt,bayes_weight
52464,thunderbirds,38,0,-3.68963
25310,boll,72,1,-3.62338
50178,<,67,1,-3.55243
52181,dahmer,31,0,-3.49180
54270,beowulf,31,0,-3.49180
25309,uwe,57,1,-3.39336
58245,deathstalker,28,0,-3.39336
56310,ajay,27,0,-3.35827
52595,welch,26,0,-3.32190
53779,tedium,26,0,-3.32190


Слова с маленьким наивным байсовским весом должны быть наоборот сильно отрицательными.\
Аномалий среди 30 последних по весу слов не наблюдается.

**e)**
### Реализовать байесовские классификаторы.


##### Модель Бернулли

Напишем функцию для векторизации текстов.

In [38]:
def vectorize(tokenized_texts, mode, bigrams=False):
    global word_to_ind
    res = [defaultdict(int) for _ in range(len(tokenized_texts))]
    for ind, text in enumerate(tokenized_texts):
        for token in text:
            if token in word_to_ind:
                res[ind][word_to_ind[token]] += 1
    if bigrams:
        for ind, text in enumerate(tokenized_texts):
            bigrams = [b for b in zip(text[:-1], text[1:])]
            for token in bigrams:
                if token in word_to_ind:
                    res[ind][word_to_ind[token]] += 1
    if mode == 'bern':
        for text in res:
            for k, v in text.items():
                if v > 0:
                    text[k] = 1
    return res

Напишем функцию для подсчёта вероятности слова в классе.

In [39]:
def prob_word_cls(ind, cls, mode='bern'):
    global word_in_pos_text, word_in_neg_text, word_to_ind, train_neg, train_pos, cnt_neg, cnt_pos, pos_words, neg_words
    if mode == 'bern':
        if cls == 'neg':
            cnt = word_in_neg_text[ind] if ind != -1 else 0
            return (1 + cnt) / (len(train_neg) + 2)
        else:
            cnt = word_in_pos_text[ind] if ind != -1 else 0
            return (1 + cnt) / (len(train_pos) + 2)
    elif mode == 'mult':
        if cls == 'neg':
            cnt = neg_words[ind] if ind != -1 else 0
            return (1 + cnt) / (cnt_neg + len(word_to_ind))
        else:
            cnt = pos_words[ind] if ind != -1 else 0
            return (1 + cnt) / (cnt_pos + len(word_to_ind))
        
# array with probability for each word
prob_w_cls_bern = [[prob_word_cls(i, cls, mode='bern') for i in range(N_words)] for cls in ('neg', 'pos')]
# логарифм вероятности, что в предложении нет ни одного слова
bern_log_prob_empty = [sum(map(lambda x: log(1 - x), prob_w_cls_bern[i])) for i in (0, 1)]
prob_w_cls_mult = [[prob_word_cls(i, cls, mode='mult') for i in range(N_words)] for cls in ('neg', 'pos')]
# print(f"CHECK PROBS: {sum(prob_w_cls_mult[0])}, {sum(prob_w_cls_mult[1])}")


Функция для вероятности класса.

In [40]:
def prob_cls(cls):
    global train_neg, train_pos
    # 0 - neg, 1 - pos
    if cls == 0:
        return len(train_neg) / (len(train_neg) + len(train_pos))
    else:
        return len(train_pos) / (len(train_neg) + len(train_pos))

 Функция для классификации одного текста.

In [41]:
def classify_bern(vectorized_text):
    # 0 - neg, 1 - pos
    classes = (0, 1)
    res = []
    for cls in classes:
        cur_res = bern_log_prob_empty[cls]
        for ind, word in enumerate(vectorized_text.keys()):
            cur_res -= log(1 - prob_w_cls_bern[cls][word])
            cur_res += log(prob_w_cls_bern[cls][word])
        cur_res += log(prob_cls(cls))
        res.append(cur_res)
    if res[0] > res[1]:
        label = 0
    else:
        label = 1
    return label
    
def classify_all(dataset, mode, return_probs=False):
    #preprocessing
    dataset = list(map(preprocessing, dataset))
    dataset = tokenize_dataset(dataset)
    dataset = vectorize(dataset, mode, bigrams=False)
    probs = []
    for i, text in enumerate(dataset):
        if mode == 'bern':
            tmp_pr = classify_bern(text)
        else:
            tmp_pr = classify_mult(text)
        probs.append(tmp_pr)
    return np.array(probs)

In [42]:
# %%timeit
my_train_labels = classify_all(train_data, mode='bern')
int_train_labels = [int(tag=='pos') for tag in train_labels]
train_corr = 0
for pair in zip(my_train_labels, int_train_labels):
    train_corr += int(pair[0] == pair[1])

train_acc = (train_corr) / len(int_train_labels)
train_acc

0.9244

In [43]:
with open(dev_texts_path, 'r', encoding='utf-8',) as inp:
    dev_data = list(map(str.strip, inp.readlines()))
with open(dev_labels_path, 'r', encoding='utf-8',) as inp:
    dev_labels = list(map(str.strip, inp.readlines()))
 
my_dev_labels = classify_all(dev_data, mode='bern')
int_dev_labels = [int(tag=='pos') for tag in dev_labels]
dev_corr = 0
for pair in zip(my_dev_labels, int_dev_labels):
    dev_corr += int(pair[0] == pair[1])
# print(dev_corr)
dev_acc = dev_corr / len(int_dev_labels)

del dev_data
del dev_labels

In [44]:
devb_acc = 1190/2000
d = {'train_acc' : train_acc, 'dev_acc': dev_acc, 'dev_b_acc': devb_acc}
accs_df = pd.DataFrame(index=[1], data = d)
accs_df

,train_acc,dev_acc,dev_b_acc
1,0.9244,0.8523,0.595


Данные по dev_b датасету были взяты при тестировании классификатора через **evaluate.py**.
Если в трейн_акк 0.99, то показываются результаты с биграммами.

##### Мультиномиальная модель

Для этой модели нам нужно поменять функцию подсчёта вероятности встретить слово в классе.\
Это реализовано в виде режима в общей функции подсчёта этой вероятности.

Теперь напишем функцию **classify_mult**

In [45]:
def classify_mult(vectorized_text):
    # 0 - neg, 1 - pos
    classes = (0, 1)
    res = []
    for cls in classes:
        cur_res = 0
        for ind, word in enumerate(vectorized_text.keys()):
            cur_res += log(prob_w_cls_mult[cls][word]) * vectorized_text[word]
        cur_res += log(prob_cls(cls))
        res.append(cur_res)
    if res[0] > res[1]:
        label = 0
    else:
        label = 1
    return label

In [46]:
my_train_labels = classify_all(train_data, mode='mult')
int_train_labels = np.array([int(tag=='pos') for tag in train_labels])
train_acc = ((my_train_labels == int_train_labels).sum() / len(int_train_labels))
train_acc

0.9148666666666667

In [47]:
with open(dev_texts_path, 'r', encoding='utf-8',) as inp:
    dev_data = list(map(str.strip, inp.readlines()))
with open(dev_labels_path, 'r', encoding='utf-8',) as inp:
    dev_labels = list(map(str.strip, inp.readlines()))

my_dev_labels = classify_all(dev_data, mode='mult')
int_dev_labels = np.array([0 if tag=='neg' else 1 for tag in dev_labels])
dev_acc = ((int_dev_labels == my_dev_labels).sum() / len(int_dev_labels))
dev_corr = 0
for pair in zip(my_dev_labels, int_dev_labels):
    dev_corr += int(pair[0] == pair[1])
dev_acc = dev_corr / len(int_dev_labels)
del dev_data
del dev_labels
dev_acc

0.846

In [48]:
devb_acc = 0
d = {'train_acc' : train_acc, 'dev_acc': dev_acc , 'dev_b_acc': 74.45}
accs_df = pd.DataFrame(index=[1], data = d)
accs_df

,train_acc,dev_acc,dev_b_acc
1,0.91487,0.846,74.45


In [49]:
print(train_acc, dev_acc)

0.9148666666666667 0.846


Данные также взяты из результатов evaluate.py

Разница в точности на тренировочной и остальных выборках происходит из-за нескольких причин:\
1. Другое распределение слов по позитивным и негативным классам.
2. Слова, которые не встречаются в тестовой выборке встретились в других выборках.

**f)** Добавить в словарь биграммы

Реализации биграмм добавлены во все методы, тут будут только точности на выборках, потому что неудобно вставлять.\
Все цифры берутся из **evaluate.py**

#### Модель Бернулли + биграммы.

In [50]:
d = {'train_acc' : 99.11, 'dev_acc': 87.36 , 'dev_b_acc': 75.85}
pd.DataFrame(index=[1], data = d)

,train_acc,dev_acc,dev_b_acc
1,99.11,87.36,75.85


#### Мультиномиальная модель + биграммы.

In [51]:
d = {'train_acc' : 98.86, 'dev_acc': 87.31 , 'dev_b_acc': 74.85}
pd.DataFrame(index=[1], data = d)

,train_acc,dev_acc,dev_b_acc
1,98.86,87.31,74.85


Триграммы появятся в research части.